In [72]:
import os
import pandas as pd
import sqlite3

In [73]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [74]:
conn = sqlite3.connect('data/external/chemphopro.db')

In [75]:
# selecting cell line
cline = 'NTERA2'  # HL60, MCF7, NTERA2
cline_sql = 'NTERA-2 clone D1'  # HL-60, MCF-7, NTERA-2 clone D1

In [76]:
# data frame with "where...." adding SQL Query parameters
ks_rel = pd.read_sql_query(f"SELECT * FROM ks_relationship WHERE cell_line = '{cline_sql}' ;", conn)

# drop duplicate entries
ks_rel = ks_rel.drop_duplicates()
ks_rel[ks_rel[['kinase', 'substrate']].duplicated()]

# add also the target protein (the protein that a phosphosite is on) to df
ks_rel['tprot'] = ks_rel['substrate'].str.replace(r'\(.*$', "")

# extract enzyme -> target protein relationships (PPIs)
ktprot_rel = ks_rel[['kinase', 'tprot']].drop_duplicates()

/var/folders/l3/vxqxpwf10_x9jp7g4_x9yy9m0000gn/T/ipykernel_70265/1028720092.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ks_rel['tprot'] = ks_rel['substrate'].str.replace(r'\(.*$', "")


In [77]:
ktprot_rel

,kinase,tprot
0,ABL1,EEF2K
1,ABL1,PARD3
2,ABL1,TNS1
3,ABL1,DDX17
4,ABL1,AFDN
...,...,...
6358,MAP3K20,BOD1L1
6359,MAP3K20,ATXN1L
6360,MAP3K20,ATAD2
6361,MAP3K20,ARHGEF9


In [78]:
# export df to csv
ks_rel.to_csv(f'data/processed/facts/pdts_{cline}.csv', index=False)
ktprot_rel.to_csv(f'data/processed/facts/pdts_tprot_{cline}.csv', index=False)

In [79]:
# write Prolog file
with open(f'models/facts/pdt_{cline}.pl', 'w') as file:
    for index, row in ks_rel.iterrows():
        var1 = "pdt('{}', '{}', '{}').".format(row['kinase'], row['substrate'], row['tprot'])
        file.write(var1 + '\n')

# write Prolog file (same file just different predicate name)
with open(f'models/facts/lint_approach/pdt_{cline}.pl', 'w') as file:
    for index, row in ks_rel.iterrows():
        var1 = "us_of('{}', '{}', '{}').".format(row['kinase'], row['substrate'], row['tprot'])
        file.write(var1 + '\n')